In [1]:
import sys
sys.path
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import bs4
import time
import os
import re
DEBUG = 1
from datetime import datetime, timedelta
import dateparser
import calendar

ModuleNotFoundError: No module named 'dateparser'

In [ ]:
def scrolldown(browser):        
    src_updated = browser.page_source
    src = ""
    while  src != src_updated:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        src = src_updated
        src_updated = browser.page_source  

In [ ]:
def connectchrome():

    # instantiate a chrome options object so you can set the size and headless preference
    options = Options()
    options.add_argument('--headless')
    options.add_argument('log-level=3')
    options.add_argument("--incognito")
    driver = webdriver.Chrome(options=options)
    driver.execute_script('window.scrollTo(0, 0);')
    # download the chrome driver from https://sites.google.com/a/chromium.org/chromedriver/downloads
    # and put it in the current directory
    chromedriver = "/home/youcef/Documents/chromedriver"
    os.environ["webdriver.chrome.driver"] = chromedriver     
    browser = webdriver.Chrome(options=options) 
    browser.get('https://www.quora.com/')
    time.sleep(2)
    return browser

In [ ]:
def extractQuestionTitle(driver):
    question_title = driver.find_element_by_xpath('.//span[contains(@class, "rendered_qtext")]').text
    return question_title

In [ ]:
def extractAnswerCount(driver):
    answer_count = driver.find_element_by_xpath("//div[@class = 'answer_count']")
    text_count = answer_count.text.split()
    count = int((text_count[0].split('+'))[0])
    return count

In [ ]:
def extractTopics(driver):
    topic_list = []
    topics = driver.find_elements_by_xpath(".//span[@class = 'TopicName TopicNameSpan']")
    for topic in topics:
        topic_list.append(topic.text)
    return topic_list

In [ ]:
def scrollDownPage(driver):
    current_scroll_position, new_height= 0, 1
    speed = 10
    while current_scroll_position <= new_height:
        current_scroll_position += speed
        driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
        new_height = driver.execute_script("return document.body.scrollHeight")

In [ ]:
def expandAllAnswers(driver):
    hidden_list = driver.find_elements_by_xpath(".//div[@class='ui_qtext_truncated ui_qtext_truncated_regular']/div[@class='ui_qtext_truncated_text']")
    #print('Number of hidden answer:',len(hidden_list))
    for hidden in hidden_list:
        hidden = hidden.find_element_by_xpath(".//a[@class='ui_qtext_more_link']")
        webdriver.ActionChains(driver).move_to_element(hidden).click(hidden).perform()

In [6]:
def extractRespond(driver):
    check_ans_count = 0

    name_list = []
    upvote_list = []
    ans_list = []
    date_list = []
    view_list = []
    error_list = []
    
    try: 
        respond_list = driver.find_elements_by_xpath(".//div[contains(@class, 'AnswerBase') and contains(@class, 'AnswerBase') and contains(@class, 'Toggle') and contains(@class, 'Answer')]")
    except Exception as e:
        error_list.append([e, question_title, check_ans_count])
        print(error_list[-1])
        return check_ans_count, name_list, upvote_list, ans_list, date_list, view_list, error_list
    
    for respond in respond_list:

        ### Name ###
        try:
            user_name = respond.find_element_by_xpath('.//a[@class = "user"]')
            name_list.append(user_name.text)
        except Exception:
            user_name = respond.find_element_by_xpath('.//span[contains(@class, "anon_user")]')
            name_list.append(user_name.text)
        except Exception as e:
            error_list.append([e, question_title, check_ans_count])
            name_list.append('')
            print(error_list[-1])
        #print(user_name.text)

        ### Date ###
        try:
            date = respond.find_element_by_xpath(".//a[@class = 'answer_permalink']")
            date_split = date.text.split('Answered ')
            question_date = date_split[1]
            date_list.append(question_date)
        except Exception as e:
            error_list.append([e, question_title, check_ans_count])
            date_list.append('')
            print(error_list[-1])

        ### Features that show after expand ###
        try:
            expanded_answer = respond.find_element_by_xpath(".//div[contains(@class,'ExpandedAnswer')]")
        except Exception as e:
            error_list.append([e, question_title, check_ans_count])
            print(error_list[-1])
            continue

        ### Answer ###
        try:
            answer = expanded_answer.find_element_by_xpath(".//div[@class='ui_qtext_expanded']/span[@class='ui_qtext_rendered_qtext']")
            ans_list.append(answer.text)
        except Exception as e:
            error_list.append([e, question_title, check_ans_count])
            ans_list.append('')
            print(error_list[-1])

        ### Answer upvote ###
        try:
            upvote = expanded_answer.find_element_by_xpath(".//a[@class='VoterListModalLink']")
            res = [int(i) for i in upvote.text.split() if i.isdigit()]
            upvote_list.append(res[0])
        except Exception:
            upvote_list.append(0)
        except Exception as e:
            error_list.append([e, question_title, check_ans_count])
            upvote_list.append(0)
            print(error_list[-1])

        ### Answer view ###
        try:
            answer_footer = expanded_answer.find_element_by_xpath(".//div[contains(@class, 'AnswerFooter')]")
            view_text = (answer_footer.text.split())[0]
            if 'k' in view_text:
                view = float((view_text.split('k'))[0]) * 1000
            elif 'm' in view_text:
                view = float((view_text.split('m'))[0]) * 1000000
            else:
                view = int(view_text)
            view_list.append(view)
        except Exception as e:
            error_list.append([e, question_title, check_ans_count])
            view_list.append(0)
            print(error_list[-1])

        check_ans_count += 1

    print("Respond collected:", check_ans_count)
    return check_ans_count, name_list, upvote_list, ans_list, date_list, view_list, error_list

In [7]:
def scrollDownPageSlower(driver):
    current_scroll_position, new_height= 0, 1
    speed = 5
    while current_scroll_position <= new_height:
        current_scroll_position += speed
        driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
        new_height = driver.execute_script("return document.body.scrollHeight")

In [8]:
def getQuestionAddedBy(driver):
    last_item = driver.find_elements_by_xpath(".//div[@class = 'pagedlist_item']")[-1]
    try:
        user_name = last_item.find_element_by_xpath(".//span[@class = 'anon_user']").text
    except:
        user_name = last_item.find_element_by_xpath(".//a[@class = 'user']").text
    return user_name

In [9]:
def extractCreateTime(driver):
    last_item = driver.find_elements_by_xpath(".//div[@class = 'pagedlist_item']")[-1]
    question_data = last_item.find_element_by_xpath(".//p[@class = 'log_action_bar']")
    question_data_split = question_data.text.split('· ')
    question_date = question_data_split[1]
    return question_date

In [10]:
def extractNoOfFollowerAndView(driver):
    stats = driver.find_elements_by_xpath(".//div[@class = 'u-flex u-flex-align--center u-padding-vert--xs u-text--gray-light pass_color_to_child_links u-sans-font-main--small']")
    no_of_follower = int((stats[0].text.split(' '))[0])
    no_of_view = (stats[1].text.split(' '))[0]
    return no_of_follower, no_of_view

In [12]:
def extractQuestion(driver, url):
    # Extract Question
    data = {}
    data['question_title'] = extractQuestionTitle(driver)
    data['answer_count'] = extractAnswerCount(driver)
    data['topic_list'] = extractTopics(driver)
    scrollDownPage(driver)  
    expandAllAnswers(driver)
    data['respond_collected'], data['name_list'], data['upvote_list'], data['ans_list'], data['date_list'], data['view_list'], data['error_list'] = extractRespond(driver)
    
    driver.get(url+'/log')
    scrollDownPageSlower(driver)
    data['question_author'] = getQuestionAddedBy(driver)
    data['create_time'] = extractCreateTime(driver)
    data['no_of_follower'], data['no_of_view'] = extractNoOfFollowerAndView(driver)
    
    json_data = json.dumps(data)
    return json_data

In [13]:
def crawlQuestionData(file):
    start_time = time.time()
    #if (DEBUG): #print ("In crawlQuestionData...")
    
    # Open question url file
    file_question_urls = open(file, mode = 'r')
    
    # we save all retrieved answers into answers.txt file
    file_answers = open("answers.txt", mode = 'a+')

    #file_users = open(os.path.join(sys.path[0]+"users.txt", mode = 'a'))
    current_lines = file_question_urls.readlines()
    i=-1
    browser= connectchrome()
    k=0
    loop=True
    #for current_line in current_lines:
    while  loop :
        current_line= current_lines[k]
        print('processing question number  : '+ str(k))
        k+=1
        if '/unanswered/' in str(current_line):
            print('answer is unanswered')
            continue     
        print ("*************************************************************************")
        if (DEBUG): print(current_line)
        question_id = current_line
        browser.get(current_line)
        json = extractQuestion(browser, current_line)
        print(json)
        
        
    
        print ("*************************************************************************")
        # In our usecase 74256 is the loop limit value
        if k>=30 :
           loop= False
    file_answers.close()
    print("--- %s seconds ---" % (time.time() - start_time))        
    browser.quit()  

In [21]:
def main():
    print(os.getcwd())
    # We merged all Questions urls crawled by 1-Questions_URLs_crawler.py into one file (quora_urls.txt)
    crawlQuestionData((os.path.join(os.getcwd()+"/quora_urls.txt")))
    return 0

if __name__ == "__main__": main()    

/home/httpak/Downloads/Quora-and-Twitter-crawler-and-user-matcher-master/1-Quora-scrapping


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


processing question number  : 0
answer is unanswered
processing question number  : 1
*************************************************************************
http://www.quora.com/Why-should-optimal-Q-values-need-to-obey-the-Bellman-equation

Siddharth Ravi
763

Respond collected: 1
{"answer_count": 1, "question_author": "Annamaneni Sriharsha", "create_time": "Jul 1, 2017", "no_of_follower": 4, "respond": 1, "topic_list": ["Bellman Equation", "Reinforcement Learning", "Equations", "Algebra"], "question_title": "Why should optimal Q values need to obey the Bellman equation?", "no_of_view": "1,274"}
*************************************************************************
processing question number  : 2
answer is unanswered
processing question number  : 3
*************************************************************************
http://www.quora.com/What-are-some-alternatives-to-the-Bellman-Equation-in-reinforcement-learning

Alex Seewald
912

Danijar Hafner
656

James MacGlashan


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[@class = 'ExpandedAnswer ExpandedContent']"}
  (Session info: headless chrome=81.0.4044.92)
